In [1]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
wpt = WordPunctTokenizer()
from string import punctuation

import nltk
from varname.helpers import Wrapper
from varname import argname

from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import ngrams
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
from tqdm import tqdm
from collections import Counter
import requests
import json
import time
import varname
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
noise = stopwords.words('russian') + list(punctuation) + list(['``',"''",'...','!','--'])
from typing import Union,Optional
from datetime import datetime
import numpy as np
trash = ['работа','компания','наш','компания','работать','год','который', 'график','новый','условие',
        'в','участие','возможность','•','умение','знание','понимание','–','—','свой','мы','задача',
         'заработный', 'плата','тк', 'рф','рабочий', 'день','требование', 'опыт','официальный', 
         'трудоустройство','оформление','высокий', 'образование','предстоять', 'заниматься','быть', 'плюс',
         'что', 'предлагать','чем', 'предстоять','требование','высокий','дружный', 'коллектив','профессиональный', 
         'рост','2', 'раз','что', 'нужно','раз', 'ум','стать','желание','грамотный', 'устный','момент','quot','ждать',
         '«', 'белый', '»','привет', 'я', 'звать','расти','интересный','достойный','вы','рада',
         'месяц','официальный', 'оформление','успешный', 'опыт','профессиональный', 'согласно','обязанность','не указано']

[nltk_data] Downloading package stopwords to C:\Users\Владислав
[nltk_data]     Дзюба\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def getAreas():
    req = requests.get('https://api.hh.ru/areas')    
    data = req.content.decode()
    req.close()
    jsObj = json.loads(data)
    areas = []
    for k in jsObj:
        for i in range(len(k['areas'])):
            if len(k['areas'][i]['areas']) != 0:                      # Если у зоны есть внутренние зоны
                for j in range(len(k['areas'][i]['areas'])):
                    areas.append([k['id'], 
                                  k['name'], 
                                  k['areas'][i]['areas'][j]['id'],
                                  k['areas'][i]['areas'][j]['name']])
            else:                                                                # Если у зоны нет внутренних зон
                areas.append([k['id'], 
                              k['name'], 
                              k['areas'][i]['id'], 
                              k['areas'][i]['name']])
     
    areas_dict = {}
    for i in areas:
        areas_dict[i[-1].lower()]=i[-2]
                
                
    return areas_dict

areas = getAreas()

In [123]:

def get_vacancys(count: int=100, city: Optional[str] = None, name: Optional[str]=None,
                 search_fields: Union[str,list[str]] = ['company_name','name','description'],
                 experience: Union[str,list[str]] = ["noExperience",
                           "between1And3",
                           "between3And6", 
                           "moreThan6"])-> pd.DataFrame:
    
    ### на один и тот же запрос допустимо лишь 2000 вакансий, поэтому ограничиваем
    if count > 2000:
        print('Максимальное значение count на категорию 2000. Ваш count будет заменен на 2000')
        count=2000
        
    # если введен поиск только по одной категории, ее нужно перевести в список, для дальнейшего итерирования
    if type(experience) is str:
        experience = [experience]
        
    df = pd.DataFrame(columns=['name','skills','experiences','employments',
                               'professional_roles','salary','city','schedules','description','url'])
    
    ########################################################### META
    # общий накопительный счет собранных вакансий
    vacancys_amount = 0
    
    # отображение параметров запроса    
    print('Параметры запроса:')  
    print()
    print(f'Специальность: {name}')   
    
    if city is None:
        print('Город: любой')
    else:
        print(f'Город: {city}')
        
    date = str(datetime.now().date())
    meta_df = pd.DataFrame({'name': f'Запрос: {name}', 'skills': 'ANY', 'experiences': [experience],
                      'employments':'ANY','professional_roles':'ANY','salary':'ANY','city':'ЛЮБОЙ' if city is None else city,
                      'schedules':'ANY','search_fields':f'ПОИСКОВЫЕ ПОЛЯ: {search_fields}', 'date': f'Дата сбора: {date}'}) 
            
    # перевод названия города в индекс города, который необходим для запроса API
    if city is not None:
        areas = getAreas()
        try:
            city = areas[city.lower()]
        except:
            print('-----------------------------------------------------------------------------------------------')
            print('Ошибка в названии города или такого города не существует. Поиск продолжится по всем городам.')
            print('-----------------------------------------------------------------------------------------------')
            city = None

    print(f'Опыт: {experience}')
    print(f'Поисковые поля: {search_fields}')
    print('-----------------------------------------------')
    print()

    # начало итераций по спсику категорий опыта 
    for grade in experience: 
        print(f'Запрос категории опыта: {grade}')
        # инициализация спиков для заполнения полей из ответа API
        list_of_id = []
                     
        list_of_skills = []
        descriptions = [ ]
        names = [] 
        schedules = []
        employments = []
        experiences = []
        professional_roles = []
        salarys = []
        citys = []
        url = []
    
        # итерации по страницам, так как вырано количество на одну страницу 100 вакансий
        # запрошенное количество делится на 100 - столько кгругов пробежит цикл с 100 ваканциями на страницу
        for i in range(int(count/100)):
            # отображение номера обрабатываемой страниницы
            if i % 2 == 0:
                print(f'Запрос страницы: {i}')
            
            page = i
            params = {
                    'text': f'NAME:{name}', 
                    'areas': 113, #113 это Россия
                    'area': city, 
                    'page': page, # Индекс страницы поиска на HH
                    'per_page': 100, # вакансий на 1 странице
                    'search_field' : search_fields,
                    'experience': grade
                }
                
            # запрос к общиму поиску вакансий
            req = requests.get('https://api.hh.ru/vacancies', params) # запрос к API
            response = req.content.decode() # декодируем  ответ, чтобы кириллица отображалась корректно
            req.close()
            
            # получение спика id каждой вакансии
            list_of_id = [i['id'] for i in json.loads(response)['items']]
            
            if len(list_of_id)==0:
                break
    
            time.sleep(0.3)
            # итерация по спсиску id и получение подробной информации по каждой вакансии
            for step, id_ in enumerate(list_of_id):
                vacancy_req = requests.get(f'https://api.hh.ru/vacancies/{id_}')
                vacancy = vacancy_req.content.decode()
                result = json.loads(vacancy)
                
                url.append(result.get('alternate_url','Не указано'))
                list_of_skills.append([skill['name'] for skill in result.get('key_skills','не указано')])
                descriptions.append(result.get('description', 'не указано'))
                names.append(result.get('name','не указано')) 
                
                schedules.append((result.get('schedule',{}) or {}).get('name','не указано'))
                employments.append((result.get('employment',{}) or {}).get('name','не указано'))
                professional_roles.append((result.get('professional_roles',{}) or [{}]) [0].get('name','не указано'))

                experiences.append((result.get('experience','не указано') or {}).get('name','не указано'))
                citys.append((result.get('area','не указано') or {}).get('name','не указано'))
                
                salary = result.get('salary',{}) or "не указано"
                if salary != "не указано":
                    salary=str((salary.get('from','не указано') or 'не указано'))+' - '+str((salary.get('to','не указано') or 'не указано')) 
                salarys.append(salary)
                time.sleep(0.3) 
                
            # итеративное пополнение счетчика вакансий    
            vacancys_amount +=len(list_of_id)  
            if i % 2 == 0:
                print('Собрано вакансий:',vacancys_amount)
                print('-------------------------------------')
        
                   
        # создание датасэта н акаждой итерации          
        df_on_iter = pd.DataFrame({'name': names,'skills':list_of_skills,
                           'experiences':experiences,'employments':employments,'professional_roles':professional_roles,
                          'salary':salarys,'city':citys, 'schedules': schedules, 'description': descriptions,'url':url})
        
        ### чистка описания 
        df_on_iter['description'] = df_on_iter['description'].apply(lambda x: re.sub(r'<\/?\w+ ?\/?>',' ',x)).\
                                        apply(lambda b: re.sub(r'^\s+|\s+$', '',b)).\
                                        apply(lambda b: re.sub(r'( )(\1)+',r'\1',b))
        
        ### в key_skills может возвращаться пустой список, если не указаны навыки
        ### заменяем на "не указано"
        df_on_iter['skills'] = df_on_iter['skills'].apply(lambda x: ', '.join(x).lower()).\
                                    apply(lambda s: 'не указано' if len(s) == 0 else s)
        
       #добавление его к общему датасэту  
        df = pd.concat([df,df_on_iter],axis=0,ignore_index=True)
        
    # перевод опыта в более привычный вид    
    df.loc[df['experiences']=='Нет опыта','experiences']='junior'
    df.loc[df['experiences']=='От 1 года до 3 лет','experiences']='1-3 years'
    df.loc[df['experiences']=='От 3 до 6 лет','experiences']='3-6 years'
    df.loc[df['experiences']=='Более 6 лет','experiences']='6 years'
    print()
    print('-------------------------------------------')
    print('ЗАВЕРШЕНО:')
    print(f'Всего собрано вакансий: {vacancys_amount}')
    
    ### дополнение мета метаданных информацией об количестве вакансий по грейдам
    meta_df = meta_df.T
    for i in df.experiences.unique():
        meta_df.loc[i] = df.experiences.value_counts()[i]
        
    meta_df = meta_df.rename(columns={0:'parameters'}).reset_index().rename(columns={'index': 'fields'})
    
    return df, meta_df

In [4]:
### функция создания сравнительных датасэтов по n-граммам и стэку
def get_words_gramm_statistic(df: pd.DataFrame,gramms: int=2,freq_amount: int=20) -> pd.DataFrame:
#     Union[str,list[str]]
    df_gramms,df_single_words,df_stack  = pd.DataFrame(),pd.DataFrame(), pd.DataFrame() 

    print(df['experiences'].value_counts())
    print()

    # итерация по уникальным грейдам в конкретном датасэте
    for n,i in enumerate(df.loc[1:]['experiences'].unique()):
        print(f'Анализируется категория: {i}')
        print('Работаем с описанием')
        
        ### обработка корпуса слов для биграмм и одиночных слов описания
        corpus = [pymorphy2_analyzer.parse(token)[0].normal_form for vacancy in df[df['experiences']==i]["description"] for token in word_tokenize(vacancy) if token not in noise]
        corpus = [word for word in corpus if word not in trash]
        
        ### выделение самых частотных n-грамм и слов описания
        freq_dict_single_words = Counter(corpus)
        single_words = freq_dict_single_words.most_common(freq_amount)#.insert(0,100)
        
        freq_dict_ngram = Counter(ngrams(corpus, gramms))
        n_gramms = freq_dict_ngram.most_common(freq_amount)
            
        ### обработка корпуса ключевых слов 
        print('Работаем со стэком')
        corpus_stack = [token for vacancy in df[df['experiences']==i]['skills'] for token in vacancy.split(', ') if token not in noise]
        corpus_stack = [word for word in corpus_stack if word not in trash]
    
        ### выделение самых частотных слов в ключевых словах
        freq_dict_stack = Counter(corpus_stack)
        stack_words = freq_dict_stack.most_common(freq_amount)
        
        ### итеративное пополнение датасэтов по грейдам
        df_gramms[i]=n_gramms
        df_single_words[i] = single_words
        df_stack[i] = stack_words
        ## список для отображения грейдов по возрастанию
        list_of_grades = [i for i in ['junior','1-3 years','3-6 years','6 years'] if i in df['experiences'].unique()]
        print('---------------------------------------')
        
    for i in list_of_grades:
        df_gramms[i] = df_gramms[i].apply(lambda x: f'{x[0]} - {x[1]}')
        df_single_words[i] = df_single_words[i].apply(lambda x: f'{x[0]} - {x[1]}')
        df_stack[i] = df_stack[i].apply(lambda x: f'{x[0]} - {x[1]}')
        
    print('ГОТОВО')
    return df_gramms[list_of_grades],\
           df_single_words[list_of_grades],\
           df_stack[list_of_grades]

In [93]:

def save_df_to_excel(*args: pd.DataFrame):
    names = argname('args')
    list_of_names = []
    for i, name_of_i in zip(args, names):
        list_of_names.append(name_of_i)
        
    name_of_excel = str(input())
    with pd.ExcelWriter(f'{name_of_excel}.xlsx') as writer:

        for n,i in enumerate(list(args)):
            i.to_excel(writer, sheet_name=list_of_names[n],na_rep='NaN')
            
            for column in i:
                if column != 'description':
                    column_width = max(i[column].astype(str).map(len).max(), len(column))
                    col_idx = i.columns.get_loc(column)+1
                    writer.sheets[list_of_names[n]].set_column(col_idx, col_idx, column_width)

        writer.save()

In [102]:
save_df_to_excel(meta,gramm,single_word_in_desc,stack,professional_roles,non_standard_professional_roles)

designers_raport_06.10.23


C:\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [56]:
gramm, single, stack = get_words_gramm_statistic(data_search_in_name,gramms=2, freq_amount=15)

1-3 years    223
3-6 years    175
junior        14
6 years       10
Name: experiences, dtype: int64

Анализируется категория: junior
Работаем с описанием
Работаем со стэком
---------------------------------------
Анализируется категория: 1-3 years
Работаем с описанием
Работаем со стэком
---------------------------------------
Анализируется категория: 3-6 years
Работаем с описанием
Работаем со стэком
---------------------------------------
Анализируется категория: 6 years
Работаем с описанием
Работаем со стэком
---------------------------------------
ГОТОВО


In [62]:
name = []
value = []
for i,item in data_search_in_name.professional_roles.value_counts().items():
    name.append(i)
    value.append(item)
    

In [70]:
non_standard_professional_roles = pd.DataFrame()
for i in data_search_in_name['professional_roles'].value_counts(normalize=True).keys()[2:]:
    non_standard_professional_roles = pd.concat([non_standard_vacancies,data_search_in_name[data_search_in_name['professional_roles']==i].iloc[:2]])
non_standard_professional_roles

,name,skills,experiences,employments,professional_roles,salary,city,schedules,description,url
81,UI/UX дизайнер,не указано,1-3 years,Полная занятость,"Программист, разработчик",не указано,Сочи,Полный день,Мы в корпоративном блоке Сбербанка разрабатыва...,https://hh.ru/vacancy/87527642
137,UX/UI Designer,"figma, html, userflow, userstory, ui, ux",1-3 years,Полная занятость,"Программист, разработчик",не указано,Алматы,Удаленная работа,​​​​​​ Требования: Опыт разработки прототипов ...,https://hh.ru/vacancy/86979514
24,UX / UI Designer,"ux, ui, figma, графический дизайн, веб-дизайн,...",1-3 years,Проектная работа,Гейм-дизайнер,300000 - 500000,Алматы,Полный день,RedFox Lab- студия мобильной разработки прилож...,https://hh.ru/vacancy/87686542
78,UX/UI дизайнера,не указано,1-3 years,Полная занятость,Гейм-дизайнер,не указано,Алматы,Полный день,« FINENEX» - Корейская IT компания сотрудничае...,https://hh.ru/vacancy/87591659
83,UX/UI Designer,"ui, ux, figma, проектирование пользовательских...",1-3 years,Полная занятость,"Арт-директор, креативный директор",не указано,Екатеринбург,Полный день,"Приветствуем, это Едем.рф - крупный информацио...",https://hh.ru/vacancy/85308476
175,Арт директор / Web UI/UX дизайнер,"figma, веб-дизайн, ui, ux, web design, графиче...",1-3 years,Проектная работа,"Арт-директор, креативный директор",100000 - не указано,Москва,Полный день,Привет! Мы творческая креативная команда 88Age...,https://hh.ru/vacancy/86601421
127,UX/UI-дизайнер,"работа с большим объемом информации, ux, ui, а...",1-3 years,Полная занятость,Аналитик,не указано,Владимир,Полный день,НАШИ ПРЕИМУЩЕСТВА: аккредитованная ИТ-компания...,https://hh.ru/vacancy/86010449
139,UX/UI проектировщик интерфейсов,"проектирование пользовательских интерфейсов, a...",1-3 years,Полная занятость,Аналитик,80000 - 120000,Москва,Полный день,Наша команда разрабатывает системы для государ...,https://hh.ru/vacancy/86225460
336,Ведущий дизайнер интерфейсов (UI/UX),"аналитическое мышление, креативность, графичес...",3-6 years,Полная занятость,Руководитель проектов,250000 - не указано,Санкт-Петербург,Полный день,Тензор – это аккредитованная ИТ-компания. Наш ...,https://hh.ru/vacancy/84959788
414,Lead UX/UI Design lead (Investment banking pla...,не указано,6 years,Полная занятость,Руководитель проектов,не указано,Москва,Полный день,Инвестиционная платформа ЮЛ Сбера – уникальная...,https://hh.ru/vacancy/87711956


In [124]:
# experience = [ "noExperience", "between1And3","between3And6", "moreThan6"]
# search_fields = ['company_name','name','description']

data_search_in_name, meta = get_vacancys(100,name='ux/ui дизайнер',search_fields='name')

Параметры запроса:

Специальность: ux/ui дизайнер
Город: любой
Опыт: ['noExperience', 'between1And3', 'between3And6', 'moreThan6']
Поисковые поля: name
-----------------------------------------------

Запрос категории опыта: noExperience
Запрос страницы: 0
Собрано вакансий: 14
-------------------------------------
Запрос категории опыта: between1And3
Запрос страницы: 0
Собрано вакансий: 114
-------------------------------------
Запрос категории опыта: between3And6
Запрос страницы: 0
Собрано вакансий: 214
-------------------------------------
Запрос категории опыта: moreThan6
Запрос страницы: 0
Собрано вакансий: 224
-------------------------------------

-------------------------------------------
ЗАВЕРШЕНО:
Всего собрано вакансий: 224


In [125]:
meta

,fields,parameters
0,name,Запрос: ux/ui дизайнер
1,skills,ANY
2,experiences,"[noExperience, between1And3, between3And6, mor..."
3,employments,ANY
4,professional_roles,ANY
5,salary,ANY
6,city,ЛЮБОЙ
7,schedules,ANY
8,search_fields,ПОИСКОВЫЕ ПОЛЯ: name
9,date,Дата сбора: 2023-10-06


In [6]:
# download = pd.read_excel('output.xlsx',sheet_name=[0,1],index_col = 0 )
# data = download[1]
# meta = download[0]

In [74]:
single_word_in_desc = single

# Эксперимент

In [63]:
df_for_recomendate = pd.concat([marketologs_in_name,ds_in_name,cmo_in_name,developers_in_name])

In [64]:
df_for_recomendate['name_for_parse'] = df_for_recomendate['name'].\
                                apply(lambda x: ' '.join([pymorphy2_analyzer.parse(token)[0].normal_form \
                                                 for token in word_tokenize(x) if token not in noise]))
df_for_recomendate['name_for_parse'] 

0             менеджер маркетинг продвижение бренд
1             помощник маркетолог удалённый работа
2                             ассистент маркетолог
3                       маркетолог-стажёр помощник
4                         junior marketing manager
                           ...                    
1277             php/go golang разработчик backend
1278             php/go golang разработчик backend
1279                  backend php developer senior
1280             php/go golang разработчик backend
1281    старший бэкенд-разработчик scala вертикаль
Name: name_for_parse, Length: 7012, dtype: object

In [66]:
tf = TfidfVectorizer()
vectors = tf.fit_transform(df_for_recomendate['name_for_parse'])

neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='euclidean')
neigh.fit(vectors)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=10)

In [84]:
marketologs_in_name

,name,skills,employments,professional_roles,salary,city,schedules,description,url,experiences
0,Менеджер по маркетингу и продвижению бренда,не указано,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",100000-170000,Москва,Полный день,Обязанности: сопровождение и развитие действую...,https://hh.ru/vacancy/87019437,junior
1,Помощник маркетолога (удаленная работа),"интернет-реклама, internet marketing, яндекс.д...",Частичная занятость,"Менеджер по маркетингу, интернет-маркетолог",19000-25000,Москва,Удаленная работа,"Привет! Меня зовут Дмитрий, я владелец консалт...",https://hh.ru/vacancy/85964259,junior
2,Ассистент маркетолога,"ms excel, маркетинг, планирование маркетинговы...",Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",None-80000,Москва,Полный день,Обязанности: запуск и контроль проведения трей...,https://hh.ru/vacancy/86974695,junior
3,Маркетолог-стажер помощник,"исполнение личных поручений руководителя, испо...",Стажировка,"Секретарь, помощник руководителя, ассистент",100000-200000,Алматы,Полный день,"Мы, рекрутинговое агентство RedLAb, занимаемся...",https://hh.ru/vacancy/86484189,junior
4,Junior marketing manager,"adobe photoshop, adobe illustrator, креативнос...",Полная занятость,Другое,60000-None,Москва,Полный день,Активно набирающая компания Happy Trade в поис...,https://hh.ru/vacancy/86180835,junior
...,...,...,...,...,...,...,...,...,...,...
4883,Трейд маркетолог,"маркетинговый анализ, анализ конкурентной сред...",Полная занятость,Руководитель отдела маркетинга и рекламы,8000000-12000000,Ташкент,Полный день,Мы - ведущая компания в сфере производства выс...,https://hh.ru/vacancy/85639866,6 years
4884,Маркетолог,не указано,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Не указано,Алматы,Полный день,"В группу компании, которая реализует услуги в ...",https://hh.ru/vacancy/85397805,6 years
4885,Маркетолог (развитие и продвижение продуктов У...,не указано,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Не указано,Москва,Полный день,Обязанности: Основная задача – развитие бренда...,https://hh.ru/vacancy/85640726,6 years
4886,Директор по маркетингу/Ведущий маркетолог,"продвижение бренда, маркетинговая стратегия, м...",Полная занятость,Руководитель отдела маркетинга и рекламы,Не указано,Санкт-Петербург,Полный день,На сегодня архитектурно-строительная компания ...,https://hh.ru/vacancy/85576389,6 years


In [82]:
role = 'Секретарь, помощник руководителя, ассистент'
normalazed_roles = [pymorphy2_analyzer.parse(token)[0].normal_form for token in word_tokenize(role) if token not in noise]
normalazed_roles =  tf.transform(normalazed_roles)

In [83]:
res = neigh.kneighbors(normalazed_roles, return_distance=True)
for i in res[1][0]:
  print(df_for_recomendate.iloc[i]['name'])

Директор по маркетингу (CMO/Chief Marketing Officer)
Директор по маркетингу / Chief Marketing Officer (CMO)
Аналитик-маркетолог (полная/частичная занятость, удаленно)
Маркетолог / интернет-маркетолог
Маркетолог/интернет-маркетолог
Руководитель отдела маркетинга/Интернет маркетолог в онлайн школу (удаленно)
Маркетолог / Интернет-маркетолог
Руководитель отдела маркетинга (главный маркетолог)
Менеджер SMM с обязанностями маркетолога
Senior NLP Data Scientist (команда SberCare.Боты)


## Балванки

In [5]:
data = pd.read_csv('D:/jupyter/re/marketologs_search_in_name_21_09_2023.csv')
data.experiences.value_counts()

1-3 years    2000
3-6 years    2000
junior        781
6 years       107
Name: experiences, dtype: int64

In [44]:
data.professional_roles.value_counts()

Менеджер по маркетингу, интернет-маркетолог             3630
Маркетолог-аналитик                                      288
SMM-менеджер, контент-менеджер                           187
Другое                                                   133
Аналитик                                                 120
Руководитель отдела маркетинга и рекламы                 115
PR-менеджер                                              100
Директор по маркетингу и PR (CMO)                         78
Менеджер по продажам, менеджер по работе с клиентами      53
Менеджер по работе с партнерами                           26
Секретарь, помощник руководителя, ассистент               25
Менеджер продукта                                         19
Event-менеджер                                            12
Копирайтер, редактор, корректор                           12
Руководитель проектов                                     11
Дизайнер, художник                                         9
Координатор отдела прода

In [39]:
value = []
name = []
for i,items in data.professional_roles.value_counts(normalize=False).items():
    name.append(i)
    value.append(items)


In [41]:
professional_roles = pd.DataFrame({'role':name,'frec':value})

In [42]:
professional_roles

,role,frec
0,"Менеджер по маркетингу, интернет-маркетолог",3630
1,Маркетолог-аналитик,288
2,"SMM-менеджер, контент-менеджер",187
3,Другое,133
4,Аналитик,120
5,Руководитель отдела маркетинга и рекламы,115
6,PR-менеджер,100
7,Директор по маркетингу и PR (CMO),78
8,"Менеджер по продажам, менеджер по работе с кли...",53
9,Менеджер по работе с партнерами,26


In [18]:
strangers_vacancys = pd.DataFrame()
for i in data['professional_roles'].value_counts(normalize=True).keys()[6:]:
    strangers_vacancys = pd.concat([strangers_vacancys,data[data['professional_roles']==i].iloc[:1]])

In [32]:
strange_vacancys = strangers_vacancys[strangers_vacancys.professional_roles!='Директор по маркетингу и PR (CMO)'].reset_index().drop('index',axis=1)

In [28]:

corpus_name = [pymorphy2_analyzer.parse(token)[0].normal_form for vacancy in data["professional_roles"] for token in word_tokenize(vacancy) if token not in noise]

corpus_name = [word for word in corpus_name if word not in trash]
print(len(corpus_name))

freq_dict_name = Counter(corpus_name).most_common(30)

13299


In [29]:
freq_dict_name

[('маркетинг', 3823),
 ('менеджер', 3792),
 ('интернет-маркетолог', 3630),
 ('маркетолог-аналитик', 288),
 ('smm-менеджер', 187),
 ('контент-менеджер', 187),
 ('руководитель', 151),
 ('другой', 133),
 ('аналитик', 123),
 ('отдел', 119),
 ('реклама', 115),
 ('pr-менеджер', 100),
 ('директор', 81),
 ('pr', 78),
 ('cmo', 78),
 ('продажа', 57),
 ('клиент', 53),
 ('партнёр', 26),
 ('секретарь', 25),
 ('помощник', 25),
 ('ассистент', 25),
 ('продукт', 19),
 ('копирайтер', 12),
 ('редактор', 12),
 ('корректор', 12),
 ('event-менеджер', 12),
 ('проект', 11),
 ('специалист', 9),
 ('дизайнер', 9),
 ('художник', 9)]

In [18]:
corpus_roles = [pymorphy2_analyzer.parse(token)[0].normal_form for vacancy in data_search_in_name["professional_roles"] for token in word_tokenize(vacancy) if token not in noise]

corpus_roles = [word for word in corpus_roles if word not in trash]
print(len(corpus_roles))

freq_dict_roles = Counter(corpus_roles)

13294


In [310]:
pd.DataFrame({'role': freq_dict_roles.most_common(30),'name':freq_dict_name.most_common(30)})

,role,name
0,"(маркетинг, 3838)","(маркетолог, 2535)"
1,"(менеджер, 3773)","(маркетинг, 920)"
2,"(интернет-маркетолог, 3635)","(менеджер, 705)"
3,"(маркетолог-аналитик, 294)","(интернет-маркетолог, 534)"
4,"(smm-менеджер, 188)","(специалист, 358)"
5,"(контент-менеджер, 188)","(реклама, 295)"
6,"(руководитель, 145)","(marketing, 294)"
7,"(другой, 134)","(manager, 262)"
8,"(аналитик, 130)","(ведущий, 250)"
9,"(отдел, 122)","(помощник, 229)"


In [19]:
set_name = set(corpus_name)
set_roles = set(corpus_roles)

In [37]:
set_name.intersection(set_roles)

{'cmo',
 'event-менеджер',
 'pr',
 'pr-менеджер',
 'smm-менеджер',
 'администратор',
 'аналитик',
 'ассистент',
 'база',
 'вэд',
 'данные',
 'дизайнер',
 'директор',
 'закупка',
 'интернет-маркетолог',
 'клиент',
 'коммерческий',
 'контент-менеджер',
 'копирайтер',
 'креативный',
 'логистика',
 'магазин',
 'маркетинг',
 'маркетолог-аналитик',
 'медицинский',
 'менеджер',
 'недвижимость',
 'отдел',
 'офис-менеджер',
 'партнёр',
 'поддержка',
 'помощник',
 'представитель',
 'продажа',
 'продукт',
 'продуктовый',
 'продюсер',
 'проект',
 'редактор',
 'реклама',
 'руководитель',
 'специалист',
 'тендер',
 'тестировщик',
 'торговый',
 'туризм',
 'финансовый'}

In [338]:
# есть в роли, но нет во названии
set_roles.difference(set_name)

{'bi-аналитик',
 'cco',
 'агент',
 'арт-директор',
 'архитектор',
 'бизнес-тренер',
 'другой',
 'зал',
 'исследователь',
 'координатор',
 'корректор',
 'менеджер/консультант',
 'метролог',
 'механик',
 'научный',
 'оператор',
 'переводчик',
 'персонал',
 'пк',
 'подбор',
 'программист',
 'разнорабочий',
 'разработчик',
 'режиссёр',
 'секретарь',
 'системный',
 'стратегия',
 'сценарист',
 'технический',
 'товаровед',
 'фармацевт-провизор',
 'художник',
 'экономист'}

In [27]:
# есть в роли, но нет во названии
in_roles_not_in_name = list(set_roles.difference(set_name))
frec = []
name_ = []
for i in in_roles_not_in_name:
    name_.append(i)
    frec.append(freq_dict_roles[i])
pd.DataFrame({'word':name_,'frec':frec}).sort_values('frec',ascending=False)

,word,frec
21,другой,134
16,корректор,19
7,секретарь,15
10,художник,6
17,персонал,5
25,координатор,4
23,экономист,4
15,товаровед,4
0,оператор,4
11,арт-директор,3


In [26]:
freq_dict = Counter(ngrams(corpus, 2))

freq_dict.most_common(30)

In [14]:
page = 1
params = {
        'text': f'NAME:Марктолог', 
        'areas': 113, #113 это Россия
        'area': 1, 
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100, # вакансий на 1 странице
    }
    
# запрос к общиму поиску вакансий
req = requests.get('https://api.hh.ru/vacancies/86914042') # запрос к API
response = req.content.decode() # декодируем  ответ, чтобы кириллица отображалась корректно
req.close()
h = json.loads(response)